In [1]:
import numpydoc as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
dataset=pd.read_csv('Social_Network_Ads.csv')
dataset

In [3]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [4]:
dataset=dataset.drop("User ID",axis=1)
dataset

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,1
1,35,20000,0,1
2,26,43000,0,0
3,27,57000,0,0
4,19,76000,0,1
...,...,...,...,...
395,46,41000,1,0
396,51,23000,1,1
397,50,20000,1,0
398,36,33000,0,1


In [5]:
dataset['Purchased'].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [6]:
dataset.columns

Index(['Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [7]:
independent=dataset[['Age', 'EstimatedSalary','Gender_Male']]
dependent=dataset[['Purchased']]
dependent

,Purchased
0,0
1,0
2,0
3,0
4,0
...,...
395,1
396,1
397,1
398,0


In [8]:
independent.shape

(400, 3)

In [9]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(independent,dependent,test_size=1/3,random_state=0)

In [10]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

In [11]:
from sklearn.tree import DecisionTreeClassifier

In [19]:
from sklearn.model_selection import GridSearchCV
param_grid={
    'criterion':['gini','entropy', 'log_loss'],
    'splitter': ['best', 'random'],
    'max_features':['sqrt', 'log2']
}
grid=GridSearchCV(DecisionTreeClassifier(),param_grid,refit=True,verbose=3,n_jobs=-1,scoring='f1_weighted')
grid.fit(x_train,y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


GridSearchCV(estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_features': ['sqrt', 'log2'],
                         'splitter': ['best', 'random']},
             scoring='f1_weighted', verbose=3)

In [29]:
re=grid.cv_results_
grid_predictions=grid.predict(x_test)
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,grid_prediction)
from sklearn.metrics import classification_report
clf_report=classification_report(y_test,grid_prediction)

In [31]:
print("the confusion Matrix:\n",cm)

the confusion Matrix:
 [[78  7]
 [10 39]]


In [33]:
print(clf_report)

              precision    recall  f1-score   support

           0       0.89      0.92      0.90        85
           1       0.85      0.80      0.82        49

    accuracy                           0.87       134
   macro avg       0.87      0.86      0.86       134
weighted avg       0.87      0.87      0.87       134



In [39]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(x_test)[:,1])

0.856782713085234

In [41]:
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_splitter,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.005043,0.001766,0.010789,0.000943,gini,sqrt,best,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.791752,0.870047,0.796284,0.906166,0.868632,0.846576,0.044996,6
1,0.003656,0.000883,0.010097,0.000688,gini,sqrt,random,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.832255,0.868752,0.849057,0.886792,0.841025,0.855576,0.019729,4
2,0.003629,0.000724,0.010868,0.001118,gini,log2,best,"{'criterion': 'gini', 'max_features': 'log2', ...",0.826263,0.888107,0.814409,0.870362,0.867097,0.853248,0.028059,5
3,0.002830,0.000555,0.009750,0.001093,gini,log2,random,"{'criterion': 'gini', 'max_features': 'log2', ...",0.794978,0.828959,0.775815,0.851527,0.849057,0.820067,0.029990,12
4,0.002776,0.000524,0.010046,0.001220,entropy,sqrt,best,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.787654,0.831253,0.814409,0.906166,0.885265,0.844949,0.044217,7
5,0.004017,0.000923,0.010328,0.001488,entropy,sqrt,random,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.867478,0.811321,0.814409,0.865054,0.811321,0.833917,0.026449,11
6,0.002718,0.001412,0.011366,0.005829,entropy,log2,best,"{'criterion': 'entropy', 'max_features': 'log2...",0.843811,0.868752,0.814409,0.925272,0.903610,0.871171,0.039889,1
7,0.004240,0.002473,0.009375,0.000489,entropy,log2,random,"{'criterion': 'entropy', 'max_features': 'log2...",0.822092,0.888107,0.758030,0.851527,0.886792,0.841310,0.048307,9
8,0.002824,0.000419,0.009479,0.000983,log_loss,sqrt,best,"{'criterion': 'log_loss', 'max_features': 'sqr...",0.849794,0.868752,0.814409,0.906166,0.867097,0.861244,0.029776,3
9,0.002628,0.000814,0.008414,0.001389,log_loss,sqrt,random,"{'criterion': 'log_loss', 'max_features': 'sqr...",0.887346,0.868752,0.795256,0.925272,0.831098,0.861545,0.044944,2


In [43]:
age_input=float(input("Age:"))
salary_input=float(input("EstimatedSalary:"))
sex_male_input=int(input("Sex Male 0 or 1:"))

Age: 56
EstimatedSalary: 900000
Sex Male 0 or 1: 1


In [47]:
grid.predict([[age_input,salary_input,sex_male_input]])

array([1], dtype=int64)

In [49]:
dir(classification_report)

['__annotations__',
 '__builtins__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__getstate__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [51]:
import pickle
filename="finalized_DecisionTreeClassifier.sav"
pickle.dump(grid,open(filename,'wb'))

In [56]:
loaded_model=pickle.load(open("finalized_DecisionTreeClassifier.sav",'rb'))
result=loaded_model.predict([[43,90000,1]])
print(result)

[1]
